# Brief

- Creating 3 files (train, valid, test) with different families for each file

In [1]:
import pandas as pd
import numpy as np


In [2]:
# Getting the list of all families Ids (FID)
df_fs = pd.read_pickle("./lists/pairs/pickles/Direct/fs-faces.pkl")
df_fs['label'] = 'fs'
df_fd = pd.read_pickle("./lists/pairs/pickles/Direct/fd-faces.pkl")
df_fd['label'] = 'fd'
df_ms = pd.read_pickle("./lists/pairs/pickles/Direct/ms-faces.pkl")
df_ms['label'] = 'ms'
df_md = pd.read_pickle("./lists/pairs/pickles/Direct/md-faces.pkl")
df_md['label'] = 'md'

df = pd.concat([df_fs, df_fd, df_ms, df_md])
df['FID'] = df['p1'].str[:5]


In [3]:
# Converting the FID to "set"
all_set = set(df['FID'].unique().tolist())

test_df = pd.read_pickle("./test_data.pkl")
test_df['FID'] = test_df['p1'].str[:5]
# Get the Test set FIDs
test_set = set(test_df['FID'].unique().tolist())

train_valid_set = all_set - test_set
valid_set = list(train_valid_set)[600:]
train_set = list(train_valid_set)[:600]

sets = {'train':train_set, 'valid':valid_set, 'test':test_set}

In [4]:
path = "./FIDs-features/"

def process(df,rType):
    df['FID'] = df['p1'].str[:5]
    df['p1'] = path+df['p1']
    df['p2'] = path+df['p2']
    df['p1'] = df['p1'].str.replace('.jpg', '.pkl')
    df['p2'] = df['p2'].str.replace('.jpg', '.pkl')
    
    for fs in sets:
        
        fs_df = df[df['FID'].isin(sets[fs])]

        sample1 = fs_df.sample(frac=1).reset_index(drop=True)[['p1','FID']]
        sample2 = fs_df.sample(frac=1).reset_index(drop=True)[['p2','FID']]
        sample2.columns = ['p2','FID2']
        unrelated = pd.concat([sample1,sample2],axis=1)
        unrelated = unrelated[unrelated['FID'] != unrelated['FID2']][['p1','p2']]

        # Related file
        p1 = fs_df['p1'].values.tolist()
        p2 = fs_df['p2'].values.tolist()
        F = pd.read_pickle(p1[0]).reshape(1,512)
        C = pd.read_pickle(p2[0]).reshape(1,512)
        related_fs_set = np.append(F,C,axis=1)

        for r1, r2 in zip(p1[1:], p2[1:]):
            F = pd.read_pickle(r1).reshape(1,512)
            C = pd.read_pickle(r2).reshape(1,512)
            temp = np.append(F,C,axis=1)
            related_fs_set = np.append(related_fs_set, temp, axis=0)
        related_df = pd.DataFrame(related_fs_set)
        related_df['label'] = rType
        related_df['class'] = 1

        # UnRelated file
        p1 = unrelated['p1'].values.tolist()
        p2 = unrelated['p2'].values.tolist()
        F = pd.read_pickle(p1[0]).reshape(1,512)
        C = pd.read_pickle(p2[0]).reshape(1,512)
        unrelated_fs_set = np.append(F,C,axis=1)

        for r1, r2 in zip(p1[1:],p2[1:]):
            F = pd.read_pickle(r1).reshape(1,512)
            C = pd.read_pickle(r2).reshape(1,512)
            temp = np.append(F,C,axis=1)
            unrelated_fs_set = np.append(unrelated_fs_set, temp, axis=0)
        unrelated_df = pd.DataFrame(unrelated_fs_set)
        unrelated_df['label'] = rType
        unrelated_df['class'] = 0
        
        mix = pd.concat([related_df, unrelated_df])
        mix = mix.sample(frac=1).reset_index(drop=True)
        mix.to_pickle("{}_{}_1024.pkl".format(fs,rType))


In [ ]:
#process(pd.read_pickle("./lists/pairs/pickles/Direct/fs-faces.pkl"),'fs')
#process(pd.read_pickle("./lists/pairs/pickles/Direct/fd-faces.pkl"),'fd')
#process(pd.read_pickle("./lists/pairs/pickles/Direct/ms-faces.pkl"),'ms')
process(pd.read_pickle("./lists/pairs/pickles/Direct/md-faces.pkl"),'md')
